In [1]:
import time

# Questão 1

Dado um conjunto de pontos bi-dimensionais, apresentados em um arquivo de texto no seguinte formato: a linha 1 contém o total de N de pontos, as linhas a N + 1 contêm as coordenadas separadas por vírgulas de cada ponto. O separador da parte fracionária será o ponto.

Exemplo: O conjunto de pontos {(1, 2), (0.5, 3), (2, 0.7), (3,5)} seria apresentado no arquivo como:

4    
1, 2    
0.5,    
3 2,    
0.7    
3, 5

a) Implemente uma função para ler o conjunto de pontos e armazenar em 2 vetores, um vetor X com as primeiras coordenadas e um vetor Y com as segundas coordenadas.

In [2]:
def ler_pontos(nome_arquivo):
    with open(nome_arquivo, "r") as f:
        n = int(f.readline().strip())
        X, Y = [], []
        for _ in range(n):
            linha = f.readline().strip().split(",")
            X.append(float(linha[0]))
            Y.append(float(linha[1]))
    return X, Y

b) Implemente uma função poly(X, k) que retorna uma matriz Ak com (k + 1) colunos de modo que os valores da coluna i são os valores de X elevados a (k + 1 -i).

Para os dados anteriores teríamos:

A1 = | 1 1 | | 0.5 1| | 2 1 | | 3 1 |

A2 = | 1 1 1 | | 0.25 0.5 1 | | 4 2 1 | | 9 3 1|

A3 = | 1 1 1 1| | 0.125 0.25 0.5 1 | | 16 4 2 1 | | 27 9 3 1 |

In [3]:
def poly(X, k):
    A = []
    for x in X:
        linha = []
        for exp in range(k, -1, -1):
            linha.append(x**exp)
        A.append(linha)
    return A

c) Implemente uma função inv(M) para inverter uma matriz qudrada M.

In [4]:
def inv(M):
    n = len(M)
    # Cria uma matriz identidade
    identidade = [[float(i == j) for j in range(n)] for i in range(n)]
    # Faz uma cópia de M para não alterar a matriz original
    mat = [linha[:] for linha in M]

    for i in range(n):
        # Pivô
        piv = mat[i][i]
        if abs(piv) < 1e-12:  # Evita divisões por zero
            # Tenta encontrar linha para trocar
            for r in range(i + 1, n):
                if abs(mat[r][i]) > abs(piv):
                    mat[i], mat[r] = mat[r], mat[i]
                    identidade[i], identidade[r] = identidade[r], identidade[i]
                    piv = mat[i][i]
                    break
        # Normaliza linha i
        for c in range(n):
            mat[i][c] /= piv
            identidade[i][c] /= piv

        # Elimina demais linhas
        for r in range(n):
            if r != i:
                fator = mat[r][i]
                for c in range(n):
                    mat[r][c] -= fator * mat[i][c]
                    identidade[r][c] -= fator * identidade[i][c]

    return identidade

d) Implente uma função ajusta(X, Y, k) que retorne P = inv(A^T A) A^T Y onde A é dado por poly(X, k).

Você pode usar funções prontas para o produto e transposição se quiser.

Os valores de P são os coeficientes do polinômio de grau k que melhor se ajustam ao conjunto de pontos segundo o critério de minimização da soma dos quadrados dos erros (SSE), onde os erros são dados por E = AP - Y.

Incluir no retorno de sua função:

* Tempo do cálculo de P
* SSE

In [5]:
def transposta(M):
    return [list(linha) for linha in zip(*M)]

In [6]:
def matmul(A, B):
    linhasA, colunasA = len(A), len(A[0])
    linhasB, colunasB = len(B), len(B[0])
    # Multiplicação possível apenas se colunasA == linhasB
    C = [[0] * colunasB for _ in range(linhasA)]
    for i in range(linhasA):
        for j in range(colunasB):
            soma = 0
            for k2 in range(colunasA):
                soma += A[i][k2] * B[k2][j]
            C[i][j] = soma
    return C

In [7]:
def ajusta(X, Y, k):
    inicio = time.time()

    A = poly(X, k)
    At = transposta(A)
    # A^T A
    M = matmul(At, A)
    # Inversão
    Minv = inv(M)
    # Converte Y para matriz coluna
    Ycol = [[y] for y in Y]
    # P = inv(A^T A) * A^T * Y
    Pmat = matmul(Minv, matmul(At, Ycol))
    # Transformar a matriz coluna em lista
    P = [row[0] for row in Pmat]

    # Cálculo do SSE
    SSE = 0
    for i in range(len(X)):
        # Predição para o ponto i
        pred = 0
        for j in range(k + 1):
            pred += A[i][j] * P[j]
        SSE += (pred - Y[i]) ** 2

    tempo_calc = time.time() - inicio
    return P, tempo_calc, SSE

e) Você receberá 3 arquvios de dados (enviados posteriormente), calcule o que se pede para cada arquivos e preencha a tabela abaixo:

| | k | Tempo de Execução (cálculo de P) | SSE | | | 1 | | | | Arquivo 1 | 2 | | | | | 10 | | | | | 1 | | | | Arquivo 2 | 2 | | | | | 10 | | | | | 1 | | | | Arquivo 3 | 2 | | | | | 10 | | |

In [8]:
def calcula_ajustes(arquivos):
    ks = [1, 2, 10]
    resultados = []
    for idx, arquivo in enumerate(arquivos, start=1):
        for k_val in ks:
            X, Y = ler_pontos(arquivo)
            P, tempo_calc, SSE = ajusta(X, Y, k_val)
            resultados.append((arquivo, k_val, tempo_calc, SSE))
    return resultados

In [9]:
def mostra_tabela(resultados):
    print("| Arquivo | k  | Tempo de Execução | SSE    |")
    for arq, k_val, tempo, sse in resultados:
        print(f"| {arq} | {k_val} | {tempo:.6f}         | {sse:.6f} |")

## OBS.: Exemplo de criação dos arquivos de teste

In [12]:
def cria_arquivos_teste():
    dados1 = """4
1, 2
0.5, 3
2, 0.7
3, 5"""

    dados2 = """5
1, 1
2, 4
3, 9
4, 16
5, 25"""

    dados3 = """6
1, 1
2, 8
3, 27
4, 64
5, 125
6, 216"""

    with open("arquivo1.txt", "w") as f:
        f.write(dados1)

    with open("arquivo2.txt", "w") as f:
        f.write(dados2)

    with open("arquivo3.txt", "w") as f:
        f.write(dados3)


# Chamada para criar os arquivos
cria_arquivos_teste()

f) Qual a complexidade de tempo teórica do seu código para o cálculo de P em função de N (número de pontos) e K (ordem do polinômio)? Os resultados obtidos na tebela estão de acordo com o que era esperado?

In [48]:
def complexidade_teorica():
    """
    A complexidade de tempo teórica do cálculo de P é dominada pelas seguintes operações:
    1. Construção da matriz A (poly): O(N * K)
    2. Cálculo de A^T A: O(N * K^2)
    3. Inversão da matriz (inv): O(K^3)
    4. Multiplicações de matrizes (matmul): O(K^3) para cada multiplicação

    Portanto, a complexidade total é:
    O(N * K + N * K^2 + K^3 + K^3) = O(N * K^2 + K^3)

    Os resultados obtidos na tabela devem ser comparados com essa complexidade teórica para verificar se estão de acordo com o esperado.
    """
    pass


# Exemplo de chamada para mostrar a tabela
arquivos = ["arquivo1.txt", "arquivo2.txt", "arquivo3.txt"]
resultados = calcula_ajustes(arquivos)
mostra_tabela(resultados)

| Arquivo | k  | Tempo de Execução | SSE    |
| arquivo1.txt | 1 | 0.000035         | 8.155593 |
| arquivo1.txt | 2 | 0.000048         | 0.492462 |
| arquivo1.txt | 10 | 0.000272         | 69860.233812 |
| arquivo2.txt | 1 | 0.000053         | 14.000000 |
| arquivo2.txt | 2 | 0.000043         | 0.000000 |
| arquivo2.txt | 10 | 0.000350         | 303658.490661 |
| arquivo3.txt | 1 | 0.000029         | 4180.800000 |
| arquivo3.txt | 2 | 0.000037         | 64.800000 |
| arquivo3.txt | 10 | 0.000334         | 88049420.673869 |
